In [1]:
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.utils.extmath import randomized_svd
from sklearn.metrics import accuracy_score, mean_squared_error

##### Generate the synthetic data

In [ ]:
# Generate data
np.random.seed(0)
X = np.random.randn(100, 10)
C_true = np.random.randn(10, 5)
Y = X @ C_true + np.random.randn(100, 5)  # Adding some noise to the data

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
Y_scaled = scaler.fit_transform(Y)


#### Convex conditioned sequential sparse learning (COSS)

In [21]:
# SVD Decomposition of C_true (Assuming known for now)
U, D, VT = randomized_svd(C_true, n_components=5)

# Compute Covariance of Y
cov_Y = np.cov(Y_scaled, rowvar=False)

# Solve eigenvalue problem for latent predictors
def solve_eigenvalue_problem(cov_matrix):
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
    return eigenvectors

Z = solve_eigenvalue_problem(cov_Y)
# Solving the eigenvalue problem for the covariance matrix of Y helps identify the principal directions (eigenvectors) in which the data varies. 
# These eigenvectors represent latent structures in the response data that can be used to project the data into a lower-dimensional space.

# Ensure Z has the correct shape
# Since X_scaled has 100 samples, we need Z to have 100 samples as well.
# We achieve this by projecting Y_scaled onto the eigenvectors.
Z_projected = Y_scaled @ Z[:, :X_scaled.shape[1]]  # Projecting to match number of samples in X_scaled

# The eigenvectors obtained from the covariance matrix represent latent structures.
# Projecting Y onto these eigenvectors helps us understand the underlying factors influencing the responses.
# By focusing on the significant eigenvectors (those associated with large eigenvalues), 
# we filter out the noise in Y, leading to a cleaner signal for the subsequent regression steps.

# Sparse left singular vector recovery using Lasso
def sparse_left_singular_vectors(W, Z_k, alpha):
    lasso = Lasso(alpha=alpha)
    lasso.fit(W, Z_k)
    return lasso.coef_
# Lasso regression is used to recover the sparse left singular vectors U^
U_hat = []
alpha = 0.1  # Lasso regularization parameter
for k in range(Z_projected.shape[1]):
    U_hat.append(sparse_left_singular_vectors(X_scaled, Z_projected[:, k], alpha))
U_hat = np.array(U_hat).T

# Adjust covariance matrices to account for measurement errors (assuming known measurement errors)
Sigma_A = np.cov(np.random.randn(10, 10), rowvar=False)  # Example additive error covariance
Sigma_W = np.cov(X_scaled, rowvar=False)
Sigma_add = Sigma_W - Sigma_A

# Recover the coefficient matrix C_hat by multipling the estimated left singular vectors U^ with the transpose of the right singular vectors VT.
C_hat = U_hat @ VT

# print("True Coefficient Matrix:\n", C_true)
# print("Recovered Coefficient Matrix:\n", C_hat)

# Compute the Mean Squared Error (MSE)
print("MSE between the true and recovered coefficient matrices:", mean_squared_error(C_true, C_hat))

MSE between the true and recovered coefficient matrices: 0.8734550299249267
